# Data Access using SparkSQL and Dataframe

## Activity : Projection and Selection Queries

In this module, you will practice how to write codes to retrieve data using Spark SQL and Dataframes API.

The complete list of Dataframe functions can be accessed from [here](https://spark.apache.org/docs/1.6.1/api/java/org/apache/spark/sql/DataFrame.html), [here](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join) and [here](https://spark.apache.org/docs/2.2.0/api/scala/index.html#org.apache.spark.sql.functions$)


In this activity, we will use HR schema as shown below
![hr](HR.gif)

### INITIALIZATION
The first section of this scipt is the initialization section. 
In this section, we are preparing Spark environment to recognize and process SQL statements.

In [1]:
from pyspark import SparkContext, SparkConf # Spark
from pyspark.sql import SparkSession # Spark SQL
from pyspark.sql.types import *

#additional 
from pyspark.sql.functions import *

sc = SparkContext.getOrCreate()

# local[*]: run Spark locally with as many working processors as logical cores on your machine.
# In the field of `master`, we use a local server with as many working processors (or threads) as possible (i.e. `local[*]`). 
# If we want Spark to run locally with 'k' worker threads, we can specify as `local[k]`.
# The `appName` field is a name to be shown on the Sparking cluster UI. 

# If there is no existing spark context, we now create a new context
if (sc is None):
    sc = SparkContext(master="local[3]", appName="Introduction to Apache Spark")
spark = SparkSession(sparkContext=sc)


## DATA STRUCTURE DEFINITION

In this section, we are preparing the data structure to match the datafiles provided as the datasources

In [2]:
#COUNTRIES TABLE
scCountries = StructType([StructField("country_id",StringType()),StructField("country_name",StringType()),StructField("region_id",IntegerType())])

#DEPARTMENTS TABLE
scDepartments = StructType([StructField("department_id",IntegerType()),
StructField("department_name",StringType()),
StructField("manager_id",IntegerType()),
StructField("location_id",IntegerType())
])

#EMPLOYEES TABLE
scEmployees = StructType([
StructField("employee_id",IntegerType()),
StructField("first_name",StringType()),
StructField("last_name",StringType()),
StructField("email",StringType()),
StructField("phone_number",StringType()),
StructField("hire_date",StringType()),
StructField("job_id",StringType()),
StructField("salary",IntegerType()),
StructField("commission_pct",FloatType()),
StructField("manager_id",IntegerType()),
StructField("department_id",IntegerType())
])

#JOBS TABLE
scJobs = StructType([
StructField("job_id",StringType()),
StructField("job_title",StringType()),
StructField("min_salary",IntegerType()),
StructField("max_salary",IntegerType())
])

#JOB_HISTORY TABLE
scJob_history = StructType([
StructField("employee_id",IntegerType()),
StructField("start_date",StringType()),
StructField("end_date",StringType()),
StructField("job_id",StringType()),
StructField("department_id",IntegerType())
])

#LOCATIONS TABLE
scLocations = StructType([
StructField("location_id",IntegerType()),
StructField("street_address",StringType()),
StructField("postal_code",StringType()),
StructField("city",StringType()),
StructField("state_province",StringType()),
StructField("country_id",StringType())
])

#REGIONS TABLE
scRegions = StructType([
StructField("region_id",IntegerType()),
StructField("region_name",StringType())
])

### DATA LOADING

In [3]:
#COUNTRIES DATA
dataCountries = sc.textFile('COUNTRIES.csv')
dataCountries = dataCountries.map(lambda x: x.split(','))
dataCountries = dataCountries.map(lambda x: [x[0],x[1], int(x[2])])

#DEPARTMENTS DATA
dataDepartments = sc.textFile('DEPARTMENTS.csv')
dataDepartments = dataDepartments.map(lambda x: x.split(','))
dataDepartments = dataDepartments.map(lambda x: [int(x[0]),x[1], int(x[2]), int(x[3])])

#EMPLOYEES DATA
dataEmployees = sc.textFile('EMPLOYEES.csv')
dataEmployees = dataEmployees.map(lambda x: x.split(','))
dataEmployees = dataEmployees.map(lambda x: [int(x[0]),x[1], x[2], \
                                             x[3],x[4], x[5], x[6], \
                                             int(x[7]),float(x[8]), int(x[9]), int(x[10])\
                                            ])

#JOBS_DATA
dataJobs = sc.textFile('JOBS.csv')
dataJobs = dataJobs.map(lambda x: x.split(','))
dataJobs = dataJobs.map(lambda x: [x[0],x[1], \
                                   int(x[2]),int(x[3])\
                                   ])

#JOB_HISTORY_DATA
dataJob_history = sc.textFile('JOB_HISTORY.csv')
dataJob_history = dataJob_history.map(lambda x: x.split(','))
dataJob_history = dataJob_history.map(lambda x: [int(x[0]),x[1], \
                                   x[2],x[3],int(x[4])\
                                   ])

#LOCATION_DATA
dataLocations = sc.textFile('LOCATIONS.csv')
dataLocations = dataLocations.map(lambda x: x.split(','))
dataLocations = dataLocations.map(lambda x: [int(x[0]),x[1], \
                                   x[2],x[3],x[4],x[5]\
                                   ])
#REGIONS DATA
dataRegions = sc.textFile('REGIONS.csv')
dataRegions = dataRegions.map(lambda x: x.split(','))
dataRegions = dataRegions.map(lambda x: [int(x[0]),x[1] ])


### PREPARING DATAFRAMES

In [4]:
dfCountries = spark.createDataFrame(dataCountries,schema=scCountries) 
dfCountries.createOrReplaceTempView("dataCountries")

dfDepartments = spark.createDataFrame(dataDepartments,schema=scDepartments) 
dfDepartments.createOrReplaceTempView("dataDepartments")

dfEmployees = spark.createDataFrame(dataEmployees,schema=scEmployees) 
dfEmployees.createOrReplaceTempView("dataEmployees")

dfJobs = spark.createDataFrame(dataJobs,schema=scJobs) 
dfJobs.createOrReplaceTempView("dataJobs")

dfJob_history = spark.createDataFrame(dataJob_history,schema=scJob_history) 
dfJob_history.createOrReplaceTempView("dataJob_history")

dfLocations = spark.createDataFrame(dataLocations,schema=scLocations) 
dfLocations.createOrReplaceTempView("dataLocations")

dfRegions = spark.createDataFrame(dataRegions,schema=scRegions) 
dfRegions.createOrReplaceTempView("dataRegions")


### PROJECTION QUERIES

#### Question 1
Display employee's first name with their last name, along with their job id and their salary.

Ensure you have the same format as expected output below 

![picture](lab2_project0.png)

In [10]:
#spark.sql()
sqlQry = spark.sql("SELECT CONCAT(first_name,' ',last_name) as FullName, job_id, salary FROM dataEmployees")
sqlQry.show()

+-----------------+----------+------+
|         FullName|    job_id|salary|
+-----------------+----------+------+
|      Steven King|   AD_PRES| 24000|
|    Neena Kochhar|     AD_VP| 17000|
|      Lex De Haan|     AD_VP| 17000|
| Alexander Hunold|   IT_PROG|  9000|
|      Bruce Ernst|   IT_PROG|  6000|
|     David Austin|   IT_PROG|  4800|
|  Valli Pataballa|   IT_PROG|  4800|
|    Diana Lorentz|   IT_PROG|  4200|
|  Nancy Greenberg|    FI_MGR| 12008|
|    Daniel Faviet|FI_ACCOUNT|  9000|
|        John Chen|FI_ACCOUNT|  8200|
|   Ismael Sciarra|FI_ACCOUNT|  7700|
|Jose Manuel Urman|FI_ACCOUNT|  7800|
|        Luis Popp|FI_ACCOUNT|  6900|
|     Den Raphaely|    PU_MAN| 11000|
|   Alexander Khoo|  PU_CLERK|  3100|
|     Shelli Baida|  PU_CLERK|  2900|
|     Sigal Tobias|  PU_CLERK|  2800|
|       Guy Himuro|  PU_CLERK|  2600|
| Karen Colmenares|  PU_CLERK|  2500|
+-----------------+----------+------+
only showing top 20 rows



In [28]:
#Put equivalence Dataframe API script in here
dfEmployees.select(concat("first_name",lit(" "), "last_name").alias("FullName"),
                   "job_id",
                   "salary").show()

+-----------------+----------+------+
|         FullName|    job_id|salary|
+-----------------+----------+------+
|      Steven King|   AD_PRES| 24000|
|    Neena Kochhar|     AD_VP| 17000|
|      Lex De Haan|     AD_VP| 17000|
| Alexander Hunold|   IT_PROG|  9000|
|      Bruce Ernst|   IT_PROG|  6000|
|     David Austin|   IT_PROG|  4800|
|  Valli Pataballa|   IT_PROG|  4800|
|    Diana Lorentz|   IT_PROG|  4200|
|  Nancy Greenberg|    FI_MGR| 12008|
|    Daniel Faviet|FI_ACCOUNT|  9000|
|        John Chen|FI_ACCOUNT|  8200|
|   Ismael Sciarra|FI_ACCOUNT|  7700|
|Jose Manuel Urman|FI_ACCOUNT|  7800|
|        Luis Popp|FI_ACCOUNT|  6900|
|     Den Raphaely|    PU_MAN| 11000|
|   Alexander Khoo|  PU_CLERK|  3100|
|     Shelli Baida|  PU_CLERK|  2900|
|     Sigal Tobias|  PU_CLERK|  2800|
|       Guy Himuro|  PU_CLERK|  2600|
| Karen Colmenares|  PU_CLERK|  2500|
+-----------------+----------+------+
only showing top 20 rows



#### Question 2
Display employee's first name with their last name, along with their email address with the domain name (@monash.edu).
Ensure you have the same format as expected output below
![result](lab2_project1.png)

In [18]:
#spark.sql()
sqlQry = spark.sql("SELECT CONCAT(first_name,' ',last_name) as FullName, CONCAT(LOWER(email),'@monash.edu') as email FROM dataEmployees")
sqlQry.show()

+-----------------+-------------------+
|         FullName|              email|
+-----------------+-------------------+
|      Steven King|   sking@monash.edu|
|    Neena Kochhar|nkochhar@monash.edu|
|      Lex De Haan| ldehaan@monash.edu|
| Alexander Hunold| ahunold@monash.edu|
|      Bruce Ernst|  bernst@monash.edu|
|     David Austin| daustin@monash.edu|
|  Valli Pataballa|vpatabal@monash.edu|
|    Diana Lorentz|dlorentz@monash.edu|
|  Nancy Greenberg|ngreenbe@monash.edu|
|    Daniel Faviet| dfaviet@monash.edu|
|        John Chen|   jchen@monash.edu|
|   Ismael Sciarra|isciarra@monash.edu|
|Jose Manuel Urman| jmurman@monash.edu|
|        Luis Popp|   lpopp@monash.edu|
|     Den Raphaely|drapheal@monash.edu|
|   Alexander Khoo|   akhoo@monash.edu|
|     Shelli Baida|  sbaida@monash.edu|
|     Sigal Tobias| stobias@monash.edu|
|       Guy Himuro| ghimuro@monash.edu|
| Karen Colmenares|kcolmena@monash.edu|
+-----------------+-------------------+
only showing top 20 rows



In [24]:
#Put equivalence Dataframe API script in here
dfEmployees.select(concat("first_name",lit(" "), "last_name").alias("FullName"),
                   concat("email",lit("@monash.edu").alias("email"))).show()

+-----------------+-------------------------------------+
|         FullName|concat(email, @monash.edu AS `email`)|
+-----------------+-------------------------------------+
|      Steven King|                     SKING@monash.edu|
|    Neena Kochhar|                  NKOCHHAR@monash.edu|
|      Lex De Haan|                   LDEHAAN@monash.edu|
| Alexander Hunold|                   AHUNOLD@monash.edu|
|      Bruce Ernst|                    BERNST@monash.edu|
|     David Austin|                   DAUSTIN@monash.edu|
|  Valli Pataballa|                  VPATABAL@monash.edu|
|    Diana Lorentz|                  DLORENTZ@monash.edu|
|  Nancy Greenberg|                  NGREENBE@monash.edu|
|    Daniel Faviet|                   DFAVIET@monash.edu|
|        John Chen|                     JCHEN@monash.edu|
|   Ismael Sciarra|                  ISCIARRA@monash.edu|
|Jose Manuel Urman|                   JMURMAN@monash.edu|
|        Luis Popp|                     LPOPP@monash.edu|
|     Den Raph

### SELECTION QUERIES

#### Question 3
Display the employees who work as *IT_PROG*

![figure](lab2_selection0.png)

In [27]:
#spark.sql()
sqlQry = spark.sql("SELECT first_name,last_name,department_id FROM dataEmployees WHERE job_id=='IT_PROG'")
sqlQry.show()

+----------+---------+-------------+
|first_name|last_name|department_id|
+----------+---------+-------------+
| Alexander|   Hunold|           60|
|     Bruce|    Ernst|           60|
|     David|   Austin|           60|
|     Valli|Pataballa|           60|
|     Diana|  Lorentz|           60|
+----------+---------+-------------+



In [29]:
#Put equivalence Dataframe API script in here
dfEmployees.select("first_name",
                   "last_name",
                   "department_id").where("job_id=='IT_PROG'").show()

+----------+---------+-------------+
|first_name|last_name|department_id|
+----------+---------+-------------+
| Alexander|   Hunold|           60|
|     Bruce|    Ernst|           60|
|     David|   Austin|           60|
|     Valli|Pataballa|           60|
|     Diana|  Lorentz|           60|
+----------+---------+-------------+



#### Question 4
Display all employees that have the salary more than 5000 and work as "Sales" (job_id beginning with 'SA')

![figure](lab2_selection1.png)

In [33]:
#spark.sql
sqlQry = spark.sql("SELECT last_name,job_id FROM dataEmployees WHERE salary>5000 AND SUBSTRING_INDEX(job_id,'_',1)=='SA'")
#search job_id by index

#sqlQry = spark.sql("SELECT last_name,job_id FROM dataEmployees WHERE salary>5000 AND job_id like 'SA%'")
#search job_id by starting word
sqlQry.show()

+---------+------+
|last_name|job_id|
+---------+------+
|  Russell|SA_MAN|
| Partners|SA_MAN|
|Errazuriz|SA_MAN|
|Cambrault|SA_MAN|
|  Zlotkey|SA_MAN|
|   Tucker|SA_REP|
|Bernstein|SA_REP|
|     Hall|SA_REP|
|    Olsen|SA_REP|
|Cambrault|SA_REP|
|  Tuvault|SA_REP|
|     King|SA_REP|
|    Sully|SA_REP|
|   McEwen|SA_REP|
|    Smith|SA_REP|
|    Doran|SA_REP|
|   Sewall|SA_REP|
|  Vishney|SA_REP|
|   Greene|SA_REP|
|  Marvins|SA_REP|
+---------+------+
only showing top 20 rows



In [37]:
#Put equivalence Dataframe API script in here
dfEmployees.select("last_name",
                   "job_id").where("5000<salary" and "SUBSTRING_INDEX(job_id,'_',1)=='SA'").show()

+---------+------+
|last_name|job_id|
+---------+------+
|  Russell|SA_MAN|
| Partners|SA_MAN|
|Errazuriz|SA_MAN|
|Cambrault|SA_MAN|
|  Zlotkey|SA_MAN|
|   Tucker|SA_REP|
|Bernstein|SA_REP|
|     Hall|SA_REP|
|    Olsen|SA_REP|
|Cambrault|SA_REP|
|  Tuvault|SA_REP|
|     King|SA_REP|
|    Sully|SA_REP|
|   McEwen|SA_REP|
|    Smith|SA_REP|
|    Doran|SA_REP|
|   Sewall|SA_REP|
|  Vishney|SA_REP|
|   Greene|SA_REP|
|  Marvins|SA_REP|
+---------+------+
only showing top 20 rows



#### Question 5

Find all employees that work in either department 50 or 60 that get paid for less than 5000

![figure](lab2_selection2.png)

In [40]:
#spark.sql()
sqlQry = spark.sql("SELECT first_name,job_id,department_id,salary FROM dataEmployees "+
                   "WHERE (department_id=='50' OR department_id=='60') AND salary<5000")
sqlQry.show()

+----------+--------+-------------+------+
|first_name|  job_id|department_id|salary|
+----------+--------+-------------+------+
|     David| IT_PROG|           60|  4800|
|     Valli| IT_PROG|           60|  4800|
|     Diana| IT_PROG|           60|  4200|
|     Julia|ST_CLERK|           50|  3200|
|     Irene|ST_CLERK|           50|  2700|
|     James|ST_CLERK|           50|  2400|
|    Steven|ST_CLERK|           50|  2200|
|     Laura|ST_CLERK|           50|  3300|
|     Mozhe|ST_CLERK|           50|  2800|
|     James|ST_CLERK|           50|  2500|
|        TJ|ST_CLERK|           50|  2100|
|     Jason|ST_CLERK|           50|  3300|
|   Michael|ST_CLERK|           50|  2900|
|        Ki|ST_CLERK|           50|  2400|
|     Hazel|ST_CLERK|           50|  2200|
|    Renske|ST_CLERK|           50|  3600|
|   Stephen|ST_CLERK|           50|  3200|
|      John|ST_CLERK|           50|  2700|
|    Joshua|ST_CLERK|           50|  2500|
|    Trenna|ST_CLERK|           50|  3500|
+----------

In [44]:
#Put equivalence Dataframe API script in here
lstQry=dfEmployees.select("first_name",
                   "job_id",
                   "department_id",
                   "salary").where("salary<5000" and "department_id==60 OR department_id==50")
lstQry.show()

+----------+--------+-------------+------+
|first_name|  job_id|department_id|salary|
+----------+--------+-------------+------+
| Alexander| IT_PROG|           60|  9000|
|     Bruce| IT_PROG|           60|  6000|
|     David| IT_PROG|           60|  4800|
|     Valli| IT_PROG|           60|  4800|
|     Diana| IT_PROG|           60|  4200|
|   Matthew|  ST_MAN|           50|  8000|
|      Adam|  ST_MAN|           50|  8200|
|     Payam|  ST_MAN|           50|  7900|
|    Shanta|  ST_MAN|           50|  6500|
|     Kevin|  ST_MAN|           50|  5800|
|     Julia|ST_CLERK|           50|  3200|
|     Irene|ST_CLERK|           50|  2700|
|     James|ST_CLERK|           50|  2400|
|    Steven|ST_CLERK|           50|  2200|
|     Laura|ST_CLERK|           50|  3300|
|     Mozhe|ST_CLERK|           50|  2800|
|     James|ST_CLERK|           50|  2500|
|        TJ|ST_CLERK|           50|  2100|
|     Jason|ST_CLERK|           50|  3300|
|   Michael|ST_CLERK|           50|  2900|
+----------

### EXPORTING RESULT TO EXTERNAL FILES

#### Question 6
Save the last query (Question 5) to csv file. Named the output folder as 'result' 

Make sure result HEADER is written to the file and file mode is OVERWRITE. 

Check your folder to verify the output file(s)

In [45]:
lstQry.coalesce(1).write.mode("overwrite").option("header", "true").csv("results")